In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
df_train = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
data=df_train.append(df_test)

In [ ]:
passenger_id=df_test['PassengerId']
df_train.set_index(['PassengerId'], inplace=True)
df_test.set_index(['PassengerId'], inplace=True)

In [ ]:
df_train.isnull().sum()

In [ ]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='median', axis=1)
Age2 = imp.fit_transform(df_train['Age'].values.reshape(1,-1))
Age2 = Age2.T
df_train['Age2']=Age2
df_train.head()

In [ ]:
df_test.isnull().sum()

In [ ]:
Age_test = imp.fit_transform(df_test['Age'].values.reshape(1,-1))
Age_test= Age_test.T
df_test['Age_Test']=Age_test
df_test.head()

In [ ]:
df_train.Embarked.value_counts()

In [ ]:
df_train.Embarked.fillna('S',inplace=True)

In [ ]:
df_test.Fare.fillna(df_test.Fare.mean(),inplace=True)

In [ ]:
df_train.drop(['Age', 'Cabin'],axis=1, inplace=True)
df_test.drop(['Age', 'Cabin'],axis=1, inplace=True)

In [ ]:
df_train.isnull().sum()

In [ ]:
df_test.isnull().sum()

In [ ]:
df_train['Sex']=df_train.Sex.apply(lambda x:0 if x=='female' else 1)
df_test['Sex']=df_test.Sex.apply(lambda x:0 if x=='female' else 1)

In [ ]:
df_train.Sex.head()

In [ ]:
df_test.Sex.head()

In [ ]:
df_train.head()

In [ ]:
import matplotlib as plt
import seaborn as sns

In [ ]:
sns.boxplot(x='Survived', y='Fare', data=df_train)

In [ ]:
df_train.shape

In [ ]:
df_train=df_train[df_train['Fare']<400]

In [ ]:
df_train.shape

In [ ]:
df_train.describe()

In [ ]:
df_train.groupby('Survived').mean()

In [ ]:
df_train.groupby('Sex').mean()

In [ ]:
df_train.corr()

In [ ]:
sns.heatmap(df_train.corr(),annot=True, cmap='PiYG')

In [ ]:
df_train['family_size']=df_train['Parch']+df_train['SibSp']+1
df_test['family_size']=df_test['Parch']+df_test['SibSp']+1

In [ ]:
def family_grp(size):
    a=''
    if(size<=1):
        a='alone'
    elif(size<=4):
        a='small'
    else:
        a='large'
    return a

df_train['family_grp']=df_train.family_size.map(family_grp)
df_test['family_grp']=df_test.family_size.map(family_grp)

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
def age_grp(age):
    a=''
    if(age<=1):
        a='infant'
    elif(age<=4):
        a='toddler'
    elif(age<=14):
        a='child'
    elif(age<=18):
        a='teenager'
    elif(age<=30):
        a='young_adult'
    elif(age<=45):
        a='adult'
    elif(age<=65):
        a='middle_age'
    else:
        a='old'
    return a
df_train['age_grp']=df_train['Age2'].map(age_grp)
df_test['age_grp']=df_test['Age_Test'].map(age_grp)

In [ ]:
df_train['fare_pp']=df_train['Fare']/df_train['family_size']
df_test['fare_pp']=df_test['Fare']/df_test['family_size']

In [ ]:
def fare_grp(fare):
    a=''
    if(fare<=4):
        a='very_low'
    elif(fare<=10):
        a='low'
    elif(fare<=20):
        a='mid'
    elif(fare<=45):
        a='high'
    else:
        a='very_high'
    return a
df_train['fare_grp']=df_train['fare_pp'].map(fare_grp)
df_test['fare_grp']=df_test['fare_pp'].map(fare_grp)

In [ ]:
df_train.head()

In [ ]:
df_train=pd.get_dummies(df_train,columns=['Embarked','family_grp','fare_grp','age_grp'],drop_first=True)
df_test=pd.get_dummies(df_test,columns=['Embarked','family_grp','fare_grp','age_grp'],drop_first=True)

In [ ]:
df_train.head()

In [ ]:
df_train.shape

In [ ]:
df_train.drop(['Name','Ticket','Fare','Age2','fare_pp','family_size'],axis=1,inplace=True)
df_test.drop(['Name','Ticket','Fare','Age_Test','fare_pp','family_size'],axis=1,inplace=True)

In [ ]:
X=df_train.drop('Survived',1)
Y=df_train['Survived']

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logreg = LogisticRegression()
logreg.fit(X, Y)
Y_pred = logreg.predict(df_test)
acc_log = round(logreg.score(X, Y) * 100, 2)
acc_log

In [ ]:
submission = pd.DataFrame({
        "PassengerId": passenger_id,
        "Survived": Y_pred
    })

In [ ]:
submission.set_index(['PassengerId'], inplace=True)

In [ ]:
# import the modules we'll need
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "data.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(submission)